In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

from sklearn.metrics.pairwise import euclidean_distances

In [2]:
# load tokenizer and model using bert romanian cased model
# https://github.com/dumitrescustefan/Romanian-Transformers
# https://huggingface.co/dumitrescustefan/bert-base-romanian-cased-v1

tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
model = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
doc1= "Sos de roșii, mozzarella, salam picant, gorgonzola, ceapă roșie, peperoncini"
doc2 = "Sos de roșii, mozzarella, șuncă, porumb, măsline, ciuperci, condimente"
doc3 = "Sos de roșii, mozzarella, salam picant, gorgonzola, hribi"
doc4 = "Sos de roșii, mozzarella, salam, condimente"
doc5 = "Sos roșii, mozzarella, sare de mare, blat crocant la foc cu lemne"

doc6 = "Penne, sos de roșii, carne de vită, condimente"
doc7 = "Penne sau spaghete, bacon, parmezan, smântână dulce, condimente"
doc8 = "Paste de casă cu sos de roșii picant ( roșii cherry, green chilli prospăt trase la tigaie în ulei de măsline extravirgin, fierte cu pulpă de roșii și fulgi de peperoncino)"
doc9 = "Spaghetti (normale/integrale), somon, smantana lichida, marar, parmezan, sare, piper"
doc10 = "Tagliatelle, pui, usturoi, ciuperci, baby spanac, parmesan, smântână lichidă, sare, piper"

docs = [doc1, doc2, doc3, doc4, doc5, doc6, doc7, doc8, doc9, doc10]

In [4]:
tokens1 = tokenizer.encode(doc1, add_special_tokens=True, return_tensors="pt")
embeddings1 = model(tokens1)[0]
tokens2 = tokenizer.encode(doc2, add_special_tokens=True, return_tensors="pt")
embeddings2 = model(tokens2)[0]
tokens3 = tokenizer.encode(doc3, add_special_tokens=True, return_tensors="pt")
embeddings3 = model(tokens3)[0]
tokens4 = tokenizer.encode(doc4, add_special_tokens=True, return_tensors="pt")
embeddings4 = model(tokens4)[0]
tokens5 = tokenizer.encode(doc5, add_special_tokens=True, return_tensors="pt")
embeddings5 = model(tokens5)[0]

tokens6 = tokenizer.encode(doc6, add_special_tokens=True, return_tensors="pt")
embeddings6 = model(tokens6)[0]
tokens7 = tokenizer.encode(doc7, add_special_tokens=True, return_tensors="pt")
embeddings7 = model(tokens7)[0]
tokens8 = tokenizer.encode(doc8, add_special_tokens=True, return_tensors="pt")
embeddings8 = model(tokens8)[0]
tokens9 = tokenizer.encode(doc9, add_special_tokens=True, return_tensors="pt")
embeddings9 = model(tokens9)[0]
tokens10 = tokenizer.encode(doc10, add_special_tokens=True, return_tensors="pt")
embeddings10 = model(tokens10)[0]

In [5]:
euclidean_distances(
    [np.mean(embeddings1.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings2.detach().numpy().squeeze(), axis=0)]
)

array([[4.893392]], dtype=float32)

In [6]:
euclidean_distances(
    [np.mean(embeddings2.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings3.detach().numpy().squeeze(), axis=0)]
)

array([[5.0934534]], dtype=float32)

In [7]:
euclidean_distances(
    [np.mean(embeddings3.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings4.detach().numpy().squeeze(), axis=0)]
)

array([[4.4306054]], dtype=float32)

In [8]:
euclidean_distances(
    [np.mean(embeddings4.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings5.detach().numpy().squeeze(), axis=0)]
)

array([[6.3659716]], dtype=float32)

In [9]:
euclidean_distances(
    [np.mean(embeddings5.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings6.detach().numpy().squeeze(), axis=0)]
)

array([[7.8989563]], dtype=float32)

In [10]:
euclidean_distances(
    [np.mean(embeddings6.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings7.detach().numpy().squeeze(), axis=0)]
)

array([[7.0283046]], dtype=float32)

In [11]:
euclidean_distances(
    [np.mean(embeddings10.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings9.detach().numpy().squeeze(), axis=0)]
)

array([[7.242952]], dtype=float32)

In [12]:
euclidean_distances(
    [np.mean(embeddings10.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings10.detach().numpy().squeeze(), axis=0)]
)

array([[4.1295309e-07]], dtype=float32)

In [13]:
euclidean_distances(
    [np.mean(embeddings1.detach().numpy().squeeze(), axis=0)],
    [np.mean(embeddings1.detach().numpy().squeeze(), axis=0)]
)

array([[0.]], dtype=float32)